In [2]:
import pandas as pd
import json
import re

In [3]:
#Cargamos el archivo json
df = pd.read_json('src/output_steam_games.json', lines=True)

In [ ]:
#Limpiamos los datos

#Eliminamos los registros que parecen venir de la concatenación de otra tabla, usando la columna publisher como proxy para identificarlos
df = df.dropna(subset=['publisher'])

#Eliminamos las columnas que no nos interesan
df = df.drop(columns=['publisher', 'url', 'reviews_url', 'discount_price', 'early_access', 'developer', 'user_id', 'items', 'items_count', 'steam_id' ])

#Llevamos los nombres a minúsculas y quitamos caracteres especiales
df['app_name'] = df['app_name'].apply(lambda x: re.sub(r'[^\w\s]', '', x.lower()) if isinstance(x, str) else x)
df['title'] = df['title'].apply(lambda x: re.sub(r'[^\w\s]', '', x.lower()) if isinstance(x, str) else x)

#Reemplazamos el &amp; de algunos géneros por &
df['genres'] = df['genres'].apply(lambda x: re.sub(r'&amp;', '&', x) if isinstance(x, str) else x)

#COnvertimos el valor free to play a 0 y también los nulos
df['price'] = df['price'].apply(lambda x: 0 if isinstance(x, str) else x)
df['price'] = df['price'].apply(lambda x: 0 if pd.isnull(x) else x)

#Quitamos el registro sin id
df.dropna(subset=['id'], inplace=True)

#Convertimos los tipos de algunas columnas para evitar errores al inferirlos
df = df.convert_dtypes()
df = df.astype({'genres': str, 'tags': str, 'specs': str, 'metascore': str})

In [37]:
#Guardamos el resultado en un archivo parquet
df.to_parquet('src/cleaned/games.parquet', index=False)